In [1]:
import sys
import os
from pathlib import Path
import tempfile
import json
import shutil
from typing import Dict, List

# Add mod directory to path for imports
mod_path = Path("../../mod")
sys.path.insert(0, str(mod_path.absolute()))

# POWSM imports
from espnet2.bin.s2t_inference import Speech2Text
from espnet2.bin.s2t_inference_language import Speech2Language
import soundfile as sf
import librosa
import numpy as np

# MFA availability check
# MFA requires Kaldi/kalpy which are conda packages
# We need to find the conda MFA, not the venv one (which lacks kalpy)
import subprocess
import shutil
import os

MFA_AVAILABLE = False
MFA_CMD = None

def find_conda_mfa():
    """Try to find MFA in conda environments."""
    # Common conda installation paths
    conda_bases = [
        os.path.expanduser("~/miniconda3"),
        os.path.expanduser("~/anaconda3"),
        os.path.expanduser("~/opt/anaconda3"),
        os.path.expanduser("~/conda"),
        "/opt/anaconda3",
        "/opt/miniconda3",
        "/usr/local/anaconda3",
        "/usr/local/miniconda3",
    ]
    
    # Check CONDA_PREFIX environment variable (current conda env)
    if "CONDA_PREFIX" in os.environ:
        conda_bases.insert(0, os.environ["CONDA_PREFIX"])
    
    # Check CONDA_DEFAULT_ENV to see if we're in a conda env
    if "CONDA_DEFAULT_ENV" in os.environ:
        env_name = os.environ["CONDA_DEFAULT_ENV"]
        if "CONDA_PREFIX" in os.environ:
            mfa_path = os.path.join(os.environ["CONDA_PREFIX"], "bin", "mfa")
            if os.path.exists(mfa_path):
                return mfa_path, os.environ["CONDA_PREFIX"]
    
    # Check common environment names
    env_names = ["aligner", "mfa", "pronunciation", "base"]
    
    for conda_base in conda_bases:
        if os.path.exists(conda_base):
            # Check if it's a conda base or envs directory
            envs_dir = os.path.join(conda_base, "envs")
            if os.path.exists(envs_dir):
                # It's a conda base, check envs
                for env_name in env_names:
                    env_path = os.path.join(envs_dir, env_name)
                    mfa_path = os.path.join(env_path, "bin", "mfa")
                    if os.path.exists(mfa_path):
                        return mfa_path, env_path
            else:
                # Might be a direct environment
                mfa_path = os.path.join(conda_base, "bin", "mfa")
                if os.path.exists(mfa_path):
                    return mfa_path, conda_base
    
    return None, None

def check_mfa_dependencies(conda_env_path):
    """Check if MFA dependencies (openfst, kaldi, kalpy) are available."""
    if not conda_env_path:
        return False, []
    
    bin_dir = os.path.join(conda_env_path, "bin")
    missing = []
    
    # Check for fstcompile (OpenFST)
    fstcompile = os.path.join(bin_dir, "fstcompile")
    if not os.path.exists(fstcompile):
        missing.append("openfst (fstcompile)")
    
    # Check for kaldi binaries (at least one)
    kaldi_bins = ["gmm-align-compiled", "gmm-latgen-faster", "fstcompile"]
    has_kaldi = any(os.path.exists(os.path.join(bin_dir, bin_name)) for bin_name in kaldi_bins)
    if not has_kaldi:
        missing.append("kaldi")
    
    # Check for kalpy (Python module - harder to check, but we'll try)
    # We can't easily check Python modules from here, but we'll catch it at runtime
    
    return len(missing) == 0, missing

# Strategy: Try conda MFA first, then check PATH
print("Checking for MFA availability...")
print("="*70)
print("MFA Setup: Hybrid Conda/Venv Configuration")
print("="*70)
print("This notebook runs in a Python venv but uses MFA from a conda environment.")
print("MFA requires Kaldi, kalpy, and OpenFST which are conda-only packages.")
print("="*70)

# First, try to find conda MFA
conda_mfa, conda_env_path = find_conda_mfa()
if conda_mfa:
    print(f"\n✓ Found conda MFA at: {conda_mfa}")
    if conda_env_path:
        print(f"  Conda environment: {conda_env_path}")
    
    # Check for required dependencies
    deps_ok, missing_deps = check_mfa_dependencies(conda_env_path)
    if not deps_ok:
        print(f"\n⚠️  Missing MFA dependencies: {', '.join(missing_deps)}")
        print("\nTo fix, run in your terminal:")
        print(f"  conda activate {os.path.basename(conda_env_path) if conda_env_path else 'aligner'}")
        print("  conda install -c conda-forge openfst kaldi kalpy")
        print("\nOr reinstall MFA with all dependencies:")
        print("  conda install -c conda-forge montreal-forced-aligner --force-reinstall")
    else:
        print("  ✓ Dependencies check passed (OpenFST, Kaldi found)")
    
    MFA_CMD = conda_mfa
    
    # Test if MFA works - check if it's executable and doesn't have kalpy errors
    mfa_works = False
    
    # Simple test: try any MFA command and check for errors
    test_commands = [
        ["--help"],
        ["version"],
    ]
    
    for test_cmd in test_commands:
        try:
            # Use conda run to ensure proper environment
            if conda_env_path:
                # Try using conda run to execute in the right environment
                env_name = os.path.basename(conda_env_path)
                result = subprocess.run(
                    ["conda", "run", "-n", env_name, "mfa"] + test_cmd,
                    capture_output=True,
                    text=True,
                    timeout=5,
                    env=os.environ.copy()
                )
            else:
                result = subprocess.run(
                    [conda_mfa] + test_cmd,
                    capture_output=True,
                    text=True,
                    timeout=5,
                    env=os.environ.copy()
                )
            
            # Check for common errors
            error_output = (result.stderr + result.stdout).lower()
            
            # Check for kalpy error
            if "kalpy" in error_output or "_kalpy" in error_output or "no module named '_kalpy'" in error_output:
                print(f"\n⚠️  MFA found but missing kalpy (Kaldi Python bindings)")
                print("   Run: conda install -c conda-forge kalpy")
                break
            
            # Check for openfst/fstcompile error
            if "fstcompile" in error_output or "openfst" in error_output or "thirdparty" in error_output:
                print(f"\n⚠️  MFA found but missing OpenFST (fstcompile)")
                print("   Run: conda install -c conda-forge openfst")
                break
            
            # If we get here and no errors, MFA should work
            if "usage" in error_output or "command" in error_output or result.returncode == 0:
                mfa_works = True
                if "version" in test_cmd and result.returncode == 0:
                    version_info = (result.stdout + result.stderr).strip()
                    if version_info:
                        print(f"\n✓ MFA version: {version_info}")
                break
        except FileNotFoundError:
            # conda command not found - try direct execution
            try:
                result = subprocess.run(
                    [conda_mfa] + test_cmd,
                    capture_output=True,
                    text=True,
                    timeout=5,
                    env=os.environ.copy()
                )
                error_output = (result.stderr + result.stdout).lower()
                if "kalpy" not in error_output and "fstcompile" not in error_output:
                    if "usage" in error_output or "command" in error_output or result.returncode == 0:
                        mfa_works = True
                        break
            except:
                pass
        except Exception as e:
            continue
    
    if mfa_works:
        MFA_AVAILABLE = True
        print("\n✓ MFA is available and ready to use!")
        # Store conda env info for later use
        if conda_env_path:
            MFA_CONDA_ENV = os.path.basename(conda_env_path)
            MFA_CONDA_ENV_PATH = conda_env_path  # Store full path for PATH manipulation
        else:
            MFA_CONDA_ENV = None
            MFA_CONDA_ENV_PATH = None
    else:
        print("\n⚠️  MFA found but may not work properly. Check dependencies above.")
        # Still store the path even if test failed, in case dependencies are installed later
        if conda_env_path:
            MFA_CONDA_ENV = os.path.basename(conda_env_path)
            MFA_CONDA_ENV_PATH = conda_env_path
        else:
            MFA_CONDA_ENV = None
            MFA_CONDA_ENV_PATH = None

# If conda MFA not found or not working, check PATH
if not MFA_AVAILABLE:
    mfa_path = shutil.which("mfa")
    if mfa_path:
        # Check if it's the venv one (which won't work)
        if ".venv" in mfa_path or "venv" in mfa_path:
            print(f"⚠️  Found MFA in venv ({mfa_path}), but it requires kalpy (conda-only)")
            print("   The venv MFA package is installed but cannot run without Kaldi/kalpy.")
            print("   ")
            print("   To use MFA, you need to:")
            print("   1. Activate your conda environment: conda activate aligner")
            print("   2. Make sure conda's bin directory is in PATH")
            print("   3. Restart this notebook")
        else:
            print(f"✓ Found MFA command at: {mfa_path}")
            MFA_CMD = mfa_path
            try:
                result = subprocess.run(
                    ["mfa", "--version"],
                    capture_output=True,
                    text=True,
                    timeout=5
                )
                if result.returncode == 0:
                    MFA_AVAILABLE = True
                    print(f"✓ MFA available: {result.stdout.strip()}")
                else:
                    print(f"⚠️  MFA command found but returned error")
                    print(f"   Error: {result.stderr if result.stderr else 'Unknown error'}")
            except Exception as e:
                print(f"⚠️  Error running MFA: {e}")

if not MFA_AVAILABLE:
    print("\n" + "="*70)
    print("⚠️  MFA not available or not working")
    print("="*70)
    print("\nMFA requires conda installation (Kaldi, kalpy, OpenFST are conda-only packages)")
    print("\n📋 Installation Instructions:")
    print("\n1. Create conda environment with MFA:")
    print("   conda create -n aligner -c conda-forge montreal-forced-aligner")
    print("\n2. Activate the environment:")
    print("   conda activate aligner")
    print("\n3. Verify installation:")
    print("   mfa --version")
    print("   # Should show MFA version without errors")
    print("\n4. Download required models:")
    print("   mfa model download dictionary english_us_mfa")
    print("   mfa model download acoustic english_mfa")
    print("\n5. For Jupyter notebook:")
    print("   Option A: Install ipykernel in conda env and use it as kernel:")
    print("     conda activate aligner")
    print("     conda install ipykernel")
    print("     python -m ipykernel install --user --name aligner --display-name 'Python (aligner)'")
    print("     # Then select 'Python (aligner)' kernel in Jupyter")
    print("\n   Option B: Keep using venv kernel, MFA will be called via subprocess")
    print("     (Current setup - should work if conda MFA is found)")
    print("\n" + "="*70)
    print("📝 Note: The notebook will still work without MFA, but timestamp alignment")
    print("   will use dummy/estimated timestamps instead of precise MFA alignments.")
    print("="*70)


Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'


/Users/umitcanevleksiz/Documents/Programming/senior/sig/exp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking for MFA availability...
MFA Setup: Hybrid Conda/Venv Configuration
This notebook runs in a Python venv but uses MFA from a conda environment.
MFA requires Kaldi, kalpy, and OpenFST which are conda-only packages.

✓ Found conda MFA at: /Users/umitcanevleksiz/miniconda3/envs/aligner/bin/mfa
  Conda environment: /Users/umitcanevleksiz/miniconda3/envs/aligner
  ✓ Dependencies check passed (OpenFST, Kaldi found)

✓ MFA is available and ready to use!


In [2]:
def parse_powsm_phones(powsm_ipa: str) -> List[str]:
    """Parse IPA phonemes from POWSM format."""
    cleaned = powsm_ipa.strip().strip('/')
    if not cleaned:
        return []
    phonemes = [p.strip('/') for p in cleaned.split('//') if p.strip('/')]
    return phonemes


def powsm_to_mfa_format(powsm_ipa: str) -> str:
    """Convert POWSM format to MFA space-separated format."""
    phones = parse_powsm_phones(powsm_ipa)
    return ' '.join(phones)


def mfa_to_powsm_format(mfa_ipa: str) -> str:
    """Convert MFA format to POWSM format."""
    if not mfa_ipa:
        return ""
    phones = mfa_ipa.strip().split()
    return '//'.join(['/' + p + '/' for p in phones])


In [3]:
# MFA utilities: validation and alignment helpers

from typing import Optional, Tuple


def validate_mfa_alignment(alignments, audio_duration: float, expected_phone_count: Optional[int] = None):
    """Validate MFA alignment quality and return metrics.

    Heuristics:
    - Long phoneme durations (>0.5s) are suspicious
    - Large gaps (>0.3s) between phonemes are suspicious
    - Coverage vs. audio duration should be close
    - Phoneme count compared to expected (if provided)
    """
    metrics = {
        "quality_score": 0.0,
        "warnings": [],
        "counts": {
            "phones": len(alignments) if alignments else 0,
            "long_segments": 0,
            "gap_segments": 0,
        },
        "coverage": None,
    }

    if not alignments:
        metrics["warnings"].append("no_alignments")
        return metrics

    durations = [max(0.0, a.get("end", 0) - a.get("start", 0)) for a in alignments]
    total_duration = sum(durations)
    long_threshold = 0.5
    long_segments = [d for d in durations if d > long_threshold]
    metrics["counts"]["long_segments"] = len(long_segments)

    # Gaps between consecutive segments
    gaps = []
    for prev, cur in zip(alignments, alignments[1:]):
        gap = max(0.0, cur.get("start", 0) - prev.get("end", 0))
        gaps.append(gap)
    gap_threshold = 0.3
    gap_segments = [g for g in gaps if g > gap_threshold]
    metrics["counts"]["gap_segments"] = len(gap_segments)

    # Coverage relative to audio duration
    if audio_duration and audio_duration > 0:
        span = max(0.0, alignments[-1].get("end", 0) - alignments[0].get("start", 0))
        metrics["coverage"] = span / audio_duration if audio_duration else None
    else:
        span = 0.0

    # Phoneme count comparison
    count_penalty = 0.0
    if expected_phone_count:
        diff_ratio = abs(len(alignments) - expected_phone_count) / max(1, expected_phone_count)
        if diff_ratio > 0.2:
            metrics["warnings"].append(f"phone_count_mismatch_{diff_ratio:.2f}")
        count_penalty = min(0.4, diff_ratio)

    # Scoring: start from 1.0 and subtract penalties
    score = 1.0
    if long_segments:
        score -= min(0.6, (len(long_segments) / max(1, len(alignments))) * 0.8)
        metrics["warnings"].append("long_segments")
    if gap_segments:
        score -= min(0.4, (len(gap_segments) / max(1, len(alignments))) * 0.6)
        metrics["warnings"].append("gappy_alignment")
    if metrics["coverage"] is not None:
        score -= min(0.4, abs(1.0 - metrics["coverage"]))
        if metrics["coverage"] < 0.6 or metrics["coverage"] > 1.2:
            metrics["warnings"].append("coverage_out_of_range")
    score -= count_penalty

    metrics["quality_score"] = max(0.0, min(1.0, score))
    return metrics


def run_mfa_alignment(
    audio_file: str,
    transcription: str,
    audio_basename: str,
    temp_base: str,
    mfa_command: str,
    env: dict,
    beam_sizes: Tuple[Tuple[int, int], ...],
    audio_duration: float,
    expected_phone_count: Optional[int] = None,
    dictionary_id: str = "english_us_mfa",
    acoustic_id: str = "english_mfa",
    label: str = "target",
):
    """Run MFA alignment for a given transcription and return alignments and diagnostics."""
    if not transcription:
        return {
            "alignments": [],
            "quality": {"quality_score": 0.0, "warnings": ["empty_transcription"]},
            "stdout": "",
            "stderr": "",
            "returncode": None,
            "textgrid_path": None,
        }

    corpus_dir = os.path.join(temp_base, f"corpus_{label}")
    output_dir = os.path.join(temp_base, f"output_{label}")
    os.makedirs(corpus_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    audio_in_corpus = os.path.join(corpus_dir, f"{audio_basename}.wav")
    shutil.copy2(audio_file, audio_in_corpus)

    lab_file = os.path.join(corpus_dir, f"{audio_basename}.lab")
    with open(lab_file, "w", encoding="utf-8") as f:
        f.write(transcription)

    result = None
    for beam, retry_beam in beam_sizes:
        cmd = [
            mfa_command,
            "align",
            corpus_dir,
            dictionary_id,
            acoustic_id,
            output_dir,
            "--clean",
            "--beam",
            str(beam),
            "--retry_beam",
            str(retry_beam),
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300, env=env)
        if result.returncode == 0:
            break

    alignments = []
    textgrid_path = os.path.join(output_dir, f"{audio_basename}.TextGrid")
    if result and result.returncode == 0 and os.path.exists(textgrid_path):
        try:
            from textgrid import TextGrid

            tg = TextGrid.fromFile(textgrid_path)
            for tier in tg.tiers:
                if tier.name.lower() in ["phones", "phone"]:
                    for interval in tier:
                        if interval.mark.strip():
                            alignments.append(
                                {
                                    "phoneme": interval.mark.strip(),
                                    "start": interval.minTime,
                                    "end": interval.maxTime,
                                }
                            )
                    break
        except ImportError:
            pass

    quality = validate_mfa_alignment(alignments, audio_duration=audio_duration, expected_phone_count=expected_phone_count)

    return {
        "alignments": alignments,
        "quality": quality,
        "stdout": result.stdout if result else "",
        "stderr": result.stderr if result else "",
        "returncode": result.returncode if result else None,
        "textgrid_path": textgrid_path if os.path.exists(textgrid_path) else None,
    }



In [4]:
# Audio trimming helper for MFA (keep original for POWSM)

import numpy as np
import soundfile as sf
import librosa


def trim_audio_for_mfa(waveform: np.ndarray, sr: int, top_db: float = 40.0):
    """Trim leading/trailing silence for MFA alignment.
    Returns trimmed audio, (start_sec, end_sec), and a flag indicating trimming was applied.
    """
    if waveform.ndim > 1:
        waveform = librosa.to_mono(waveform.T)
    non_silent, idx = librosa.effects.trim(waveform, top_db=top_db)
    if idx is None or len(non_silent) == 0:
        return waveform, (0.0, len(waveform) / sr), False
    start_sample, end_sample = idx
    start_sec = start_sample / sr
    end_sec = end_sample / sr
    # Only consider it trimmed if we actually removed something noticeable (>10ms)
    trimmed = (start_sample > sr * 0.01) or (len(waveform) - end_sample > sr * 0.01)
    return non_silent, (start_sec, end_sec), trimmed



In [5]:
# Load POWSM models

device = "cpu"  # Change to "cuda" if GPU available

print("Loading POWSM models...")

# Language detection model
print("  - Loading language detection model...")
s2lang = Speech2Language.from_pretrained(
    "espnet/powsm",
    device=device,
    nbest=1,
    first_lang_sym="<afr>",
    last_lang_sym="<zul>"
)

# Phone Recognition model
print("  - Loading Phone Recognition (PR) model...")
s2t_pr = Speech2Text.from_pretrained(
    "espnet/powsm",
    device=device,
    lang_sym="<eng>",
    task_sym="<pr>",
)

# ASR model (for G2P)
print("  - Loading ASR model...")
s2t_asr = Speech2Text.from_pretrained(
    "espnet/powsm",
    device=device,
    lang_sym="<eng>",
    task_sym="<asr>",
)

# G2P model
print("  - Loading G2P model...")
s2t_g2p = Speech2Text.from_pretrained(
    "espnet/powsm",
    device=device,
    lang_sym="<eng>",
    task_sym="<g2p>",
)

print("✓ All POWSM models loaded!")


Loading POWSM models...
  - Loading language detection model...


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 74329.44it/s]


  - Loading Phone Recognition (PR) model...


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 75089.84it/s]


  - Loading ASR model...


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 78713.48it/s]


  - Loading G2P model...


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 84368.18it/s]


✓ All POWSM models loaded!


In [6]:
# CONFIG: Set audio file and text file paths

AUDIO_FILE = Path("./audio/powsm/12/umit12-r.wav")
TEXT_FILE = Path("./audio/powsm/12/text")

if not AUDIO_FILE.exists():
    raise FileNotFoundError(f"Audio file not found: {AUDIO_FILE}")

if not TEXT_FILE.exists():
    raise FileNotFoundError(f"Text file not found: {TEXT_FILE}")

with open(TEXT_FILE, 'r', encoding='utf-8') as f:
    target_text = f.read().strip()

audio_file = str(AUDIO_FILE)
audio_basename = AUDIO_FILE.stem

print(f"Audio: {AUDIO_FILE}")
print(f"Text: {target_text}...")

def map_errors_to_timestamps(
    operations: List,
    actual_phones: List[str],
    target_phones: List[str],
    mfa_alignments_actual: List[Dict],
    mfa_alignments_target: List[Dict],
    sample_rate: int
) -> List[Dict]:
    errors_with_timestamps = []

    def pick_alignments(op_type: str):
        if op_type == "delete":
            return mfa_alignments_target or mfa_alignments_actual or []
        return mfa_alignments_actual or mfa_alignments_target or []

    def fallback_stamp(pos: int):
        start_time = pos * 0.1
        end_time = (pos + 1) * 0.1
        return {
            "timestamp_seconds": {"start": start_time, "end": end_time},
            "timestamp_samples": {"start": int(start_time * sample_rate), "end": int(end_time * sample_rate)},
        }

    for op in operations:
        op_type = op[0]
        pos = op[1]
        alignments = pick_alignments(op_type)
        error_dict = {"type": op_type, "position": pos}

        if op_type == "substitute":
            error_dict["expected"] = op[2] if len(op) > 2 else None
            error_dict["actual"] = op[3] if len(op) > 3 else None
            if pos < len(alignments):
                align = alignments[pos]
                error_dict["timestamp_seconds"] = {"start": align["start"], "end": align["end"]}
                error_dict["timestamp_samples"] = {
                    "start": int(align["start"] * sample_rate),
                    "end": int(align["end"] * sample_rate),
                }
            else:
                error_dict.update(fallback_stamp(pos))
        elif op_type == "insert":
            error_dict["actual"] = op[2] if len(op) > 2 else None
            if pos > 0 and pos <= len(alignments):
                if pos < len(alignments):
                    prev_end = alignments[pos - 1]["end"]
                    next_start = alignments[pos]["start"]
                    mid_time = (prev_end + next_start) / 2
                    error_dict["timestamp_seconds"] = {"start": mid_time - 0.05, "end": mid_time + 0.05}
                else:
                    last_end = alignments[-1]["end"]
                    error_dict["timestamp_seconds"] = {"start": last_end, "end": last_end + 0.1}
            else:
                error_dict["timestamp_seconds"] = {"start": 0.0, "end": 0.1}
            error_dict["timestamp_samples"] = {
                "start": int(error_dict["timestamp_seconds"]["start"] * sample_rate),
                "end": int(error_dict["timestamp_seconds"]["end"] * sample_rate),
            }
        elif op_type == "delete":
            error_dict["expected"] = op[2] if len(op) > 2 else None
            if pos < len(alignments):
                align = alignments[pos]
                error_dict["timestamp_seconds"] = {"start": align["start"], "end": align["end"]}
                error_dict["timestamp_samples"] = {
                    "start": int(align["start"] * sample_rate),
                    "end": int(align["end"] * sample_rate),
                }
            else:
                error_dict.update(fallback_stamp(pos))
        else:
            error_dict.update(fallback_stamp(pos))

        errors_with_timestamps.append(error_dict)

    return errors_with_timestamps


Audio: audio/powsm/12/umit12-r.wav
Text: The weather is rather warm this Thursday. I think we should go to the theater together. Thank you for thinking about this thoroughly....


In [8]:
# Load audio

try:
    speech, rate = sf.read(audio_file)
except Exception as e:
    print(f"soundfile failed, trying librosa...")
    speech, rate = librosa.load(audio_file, sr=16000)

# Trim for MFA alignment only (keep original for POWSM inference)
speech_mfa, (trim_start, trim_end), was_trimmed = trim_audio_for_mfa(speech, rate, top_db=40.0)
trimmed_duration = len(speech_mfa) / rate

audio_duration = len(speech) / rate
print(f"Audio loaded: {audio_duration:.2f}s @ {rate}Hz (MFA uses trimmed {trimmed_duration:.2f}s, trimmed={was_trimmed})")

# Step 1: Extract actual pronunciation from audio

print("\nStep 1: Phone Recognition...")
result_pr = s2t_pr(speech, text_prev="<na>")
pred_pr = result_pr[0][0]

if "<notimestamps>" in pred_pr:
    pred_pr = pred_pr.split("<notimestamps>")[1].strip()
else:
    pred_pr = pred_pr.strip()

actual_phones = parse_powsm_phones(pred_pr)
print(f"Detected {len(actual_phones)} phones")

# Step 2: Generate target pronunciation from text

print("\nStep 2: G2P...")
result_asr = s2t_asr(speech, text_prev="<na>")
pred_asr = result_asr[0][0]
if "<notimestamps>" in pred_asr:
    pred_asr = pred_asr.split("<notimestamps>")[1].strip()
else:
    pred_asr = pred_asr.strip()

result_g2p = s2t_g2p(speech, text_prev=target_text)
pred_g2p = result_g2p[0][0]
if "<notimestamps>" in pred_g2p:
    pred_g2p = pred_g2p.split("<notimestamps>")[1].strip()
else:
    pred_g2p = pred_g2p.strip()

target_phones = parse_powsm_phones(pred_g2p)
print(f"Target: {len(target_phones)} phones")

# Step 3: MFA alignment

print("\nStep 3: MFA Alignment...")
mfa_alignments_target = []
mfa_alignments_actual = []
mfa_quality_target = {}
mfa_quality_actual = {}
beam_sizes = ((400, 1600))

if MFA_AVAILABLE:
    with tempfile.TemporaryDirectory() as temp_base:
        try:
            mfa_command = MFA_CMD if MFA_CMD else "mfa"
            env = os.environ.copy()
            if 'MFA_CONDA_ENV_PATH' in globals() and MFA_CONDA_ENV_PATH:
                conda_bin = os.path.join(MFA_CONDA_ENV_PATH, "bin")
                env["PATH"] = conda_bin + os.pathsep + env.get("PATH", "")

            # Write trimmed audio for MFA use
            trimmed_audio_path = os.path.join(temp_base, "audio_for_mfa.wav")
            sf.write(trimmed_audio_path, speech_mfa, rate)
            audio_used_for_mfa = trimmed_audio_path if was_trimmed else audio_file

            target_result = run_mfa_alignment(
                audio_file=audio_used_for_mfa,
                transcription=target_text,
                audio_basename=audio_basename,
                temp_base=temp_base,
                mfa_command=mfa_command,
                env=env,
                beam_sizes=beam_sizes,
                audio_duration=trimmed_duration if was_trimmed else audio_duration,
                expected_phone_count=len(target_phones),
                label="target",
            )
            mfa_alignments_target = target_result["alignments"]
            mfa_quality_target = target_result["quality"]
            if mfa_alignments_target:
                print(f"Aligned target {len(mfa_alignments_target)} phones")
            else:
                print("No target alignment returned")

            actual_result = run_mfa_alignment(
                audio_file=audio_used_for_mfa,
                transcription=pred_asr,
                audio_basename=audio_basename,
                temp_base=temp_base,
                mfa_command=mfa_command,
                env=env,
                beam_sizes=beam_sizes,
                audio_duration=trimmed_duration if was_trimmed else audio_duration,
                expected_phone_count=len(actual_phones),
                label="actual",
            )
            mfa_alignments_actual = actual_result["alignments"]
            mfa_quality_actual = actual_result["quality"]
            if mfa_alignments_actual:
                print(f"Aligned actual {len(mfa_alignments_actual)} phones")
            else:
                print("No actual alignment returned")
        except Exception as e:
            print(f"MFA error: {e}")
else:
    print("MFA not available; using estimated timestamps")

# Step 4: Compare actual vs target

print("\nStep 4: Error Detection...")
sys.path.insert(0, str(mod_path / "assessment"))
from edit_distance import edit_operations

operations = edit_operations(actual_phones, target_phones)
# Operations format:
# - insert: (op_type, pos_in_actual, actual_value) - extra element you said
# - delete: (op_type, pos_in_target, target_value) - missing element you should have said
# - substitute: (op_type, pos_in_actual, expected, actual_value) - changed FROM expected TO actual_value
substitutes = [op for op in operations if op[0] == "substitute"]
inserts = [op for op in operations if op[0] == "insert"]
deletes = [op for op in operations if op[0] == "delete"]

total_phonemes = len(target_phones)
if total_phonemes == 0:
    score = 1.0 if len(actual_phones) == 0 else 0.0
else:
    error_cost = sum(
        1 if op[0] == "delete" else
        1 if op[0] == "insert" else
        2 if op[0] == "substitute" else 0
        for op in operations
    )
    max_cost = total_phonemes * 2
    score = max(0.0, 1.0 - (error_cost / max_cost))

print(f"Errors: {len(operations)}, Score: {score:.2%}")

# Step 5: Map errors to timestamps

errors_with_timestamps = map_errors_to_timestamps(
    operations,
    actual_phones,
    target_phones,
    mfa_alignments_actual,
    mfa_alignments_target,
    rate,
)

# Step 6: Word-level comparison

import re
def tokenize_words(text: str):
    if not text:
        return []
    return re.findall(r"[A-Za-z']+", text.lower())

target_words = tokenize_words(target_text)
actual_words = tokenize_words(pred_asr)
word_operations = edit_operations(actual_words, target_words)

if len(target_words) == 0:
    word_error_rate = 0.0 if len(actual_words) == 0 else 1.0
else:
    word_error_rate = len(word_operations) / len(target_words)

print(f"\nWord Error Rate: {word_error_rate:.2%}")

# Save results

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

assessment_result = {
    "metadata": {
        "audio_file": os.path.basename(audio_file),
        "audio_path": audio_file,
        "audio_basename": audio_basename,
        "audio_duration_seconds": audio_duration,
        "sample_rate": rate,
        "timestamp": timestamp,
        "trim_for_mfa_seconds": {
            "start": trim_start,
            "end": trim_end,
            "duration": trimmed_duration,
            "applied": was_trimmed,
        },
    },    
    "statistics": {
        "total_errors": len(operations),
        "substitutions": len(substitutes),
        "insertions": len(inserts),
        "deletions": len(deletes),
        "actual_phone_count": len(actual_phones),
        "target_phone_count": len(target_phones),
    },
    "word_level": {
        "target_words": target_words,
        "actual_words": actual_words,
        "errors": word_operations,
        "statistics": {
            "total_errors": len(word_operations),
            "target_word_count": len(target_words),
            "actual_word_count": len(actual_words),
            "word_error_rate": word_error_rate,
        },
    },
    "target_text": target_text,
    "actual_text": pred_asr,
    "actual_ipa_powsm": pred_pr,
    "target_ipa_powsm": pred_g2p,
    "actual_phones": actual_phones,
    "target_phones": target_phones,
    "score": score,
    "errors": errors_with_timestamps,
    "mfa_alignments": mfa_alignments_actual or mfa_alignments_target,
    "mfa_alignments_actual": mfa_alignments_actual,
    "mfa_alignments_target": mfa_alignments_target,
    "mfa": {
        "available": MFA_AVAILABLE,
        "target": {
            "quality_score": mfa_quality_target.get("quality_score"),
            "warnings": mfa_quality_target.get("warnings", []),
            "metrics": mfa_quality_target,
        },
        "actual": {
            "quality_score": mfa_quality_actual.get("quality_score"),
            "warnings": mfa_quality_actual.get("warnings", []),
            "metrics": mfa_quality_actual,
        },
    },

}

output_dir = Path("results")
output_dir.mkdir(exist_ok=True)
output_filename = f"{audio_basename}_{timestamp}.json"
output_file = output_dir / output_filename

assessment_result["metadata"]["output_file"] = str(output_file)

def _json_default(obj):
    try:
        import numpy as np  # local import to avoid dependency if not needed
        if isinstance(obj, (np.bool_,)):
            return bool(obj)
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating,)):
            return float(obj)
    except Exception:
        pass
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(assessment_result, f, indent=2, ensure_ascii=False, default=_json_default)

print(f"\nResults saved to: {output_file}")
print(f"Pronunciation Score: {score:.2%}")
print(f"Word Error Rate: {word_error_rate:.2%}")


Audio loaded: 20.00s @ 16000Hz (MFA uses trimmed 9.15s, trimmed=True)

Step 1: Phone Recognition...
Detected 86 phones

Step 2: G2P...
Target: 85 phones

Step 3: MFA Alignment...
Aligned target 77 phones
Aligned actual 68 phones

Step 4: Error Detection...
Errors: 8, Score: 93.53%

Word Error Rate: 21.74%

Results saved to: results/umit12-r_20251211_115310.json
Pronunciation Score: 93.53%
Word Error Rate: 21.74%
